###   Simplified Hello UFO Dataflow
![Hello UFO](https://docs.google.com/drawings/d/16GcbaiyxkE8eKsKiFoIYOMe_VtuDLU7lIFXXUIktAbo/pub?w=1200&h=597)

In [1]:
import pandas as pd
import dataflowkit.datasets as D
import dataflowkit.recipes as R

### Implement the logic of recipes

In [2]:
class Formatter(R.BaseRecipe):
    def execute(self, ins, outs):
        ufo_observation = ins['ufo_observation'].load()
        rows = ufo_observation.split('\n')
        location = ''
        shape = ''
        duration = ''
        description = ''
        for row in rows:
            if row.lower().startswith('location'):
                location = ':'.join(row.split(':')[1:]).strip()
            elif row.lower().startswith('shape'):
                shape = ':'.join(row.split(':')[1:]).strip()
            elif row.lower().startswith('duration'):
                duration = ':'.join(row.split(':')[1:]).strip()
            elif row.lower().startswith('occurred'):
                pass
            elif row.lower().startswith('reported'):
                pass
            elif row.lower().startswith('posted'):
                pass
            else:
                description += row
        formatted = [{'location': location, 'shape': shape, 'duration': duration, 'description': description}]
        outs['formatted'].save(formatted)

In [3]:
class ColorParser(R.BaseRecipe):
    def execute(self, ins, outs):
        formatted = ins['formatted'].load()
        descriptions = formatted[0]['description'].split(' ')
        colors = ['whiteish', 'white', 'red', 'yellow'] # ...
        parse_colors = set()
        for c in colors:
            if c in descriptions:
                parse_colors.add(c)
        colors = [{'color': c}for c in parse_colors]
        outs['color'].save(colors)

In [4]:
class TypeClassifier(R.BaseRecipe):
    def execute(self, ins, outs):
        # dummy implementation
        outs['type'].save([
                {'type': 'diskoid', 'probability': 0.7}, 
                {'type': 'cigar', 'probability': 0.2},
                {'type': 'spheroid', 'probability': 0.1}
            ])

### Declare recipes and datasets

In [5]:
ufo_observation = D.InMemory()
formatted = D.InMemory()
color = D.InMemory()
type = D.InMemory()

formatter = Formatter()
color_parser = ColorParser()
type_classifier = TypeClassifier()

### Execute with procedure
It is recommeded to store the precedures inside a graph class. It is easier to maintain.

In [6]:
report = """
Occurred : 1/11/2017 18:22 (Entered as : 01/11/2017 19:22)
Reported: 1/11/2017 8:28:18 PM 20:28
Posted: 1/12/2017
Location: Hinesville, GA
Shape: Disk
Duration:.30
1/11/2017 at 6:22 pm I saw two circular flying sausers very bright whiteish in color, and they flew in one on top of the other stoped i

At 6:05 P.M on 1/11/2017 I Joshua Holland was sitting on my front porch looking at 5 jets flying in the sky. After 15 minutes they flew off and could no longer see them in the sky. At exactly 6:22 P.M I looked into the sky, and to my left I saw not one but two very bright disc shaped craft flying faster then anything iv ever seen in our skies one on top of the other, whitish in color. They were flying in what looked like the direction of the military base close by my house. 

These craft stopped in mid air, hovered for just a moment then started flying back into the opposite direction were they originally came from. After three seconds the craft on top flew in front of there craft below. The second craft admitted a flash of light twice then both craft vanished into thin air. I never took my eyes off of them the entire time. 

Once they vanished I saw a jet fly directly into the same path were the two crafts vanished. I remember seeing the two crafts flying in a very odd manner. They were not fly straight like a plain or jet would, they flew in a very fast circular motion. 

I have never seen anything quite like this before. I now am a firm believer that there things in our skies that we can’t explain.


((NUFORC Note: We spoke via telephone with the witness, and he sounded to us to be quite sincere and sober-minded. We suspect that he is a reliable, and skilled, witness. PD))
"""

In [7]:
ufo_observation.save(report)
formatter.execute(ins=dict(ufo_observation=ufo_observation), outs=dict(formatted=formatted))
color_parser.execute(ins=dict(formatted=formatted), outs=dict(color=color))
type_classifier.execute(ins=dict(color=color), outs=dict(type=type))

### Check data integrity

In [8]:
from IPython.display import display
display(pd.DataFrame.from_dict(formatted.load()))
display(pd.DataFrame.from_dict(color.load()))
display(pd.DataFrame.from_dict(type.load()))

,description,duration,location,shape
0,1/11/2017 at 6:22 pm I saw two circular flying...,.30,"Hinesville, GA",Disk


,color
0,whiteish


,probability,type
0,0.7,diskoid
1,0.2,cigar
2,0.1,spheroid
